In [1]:
!pip install streamlit sounddevice assemblyai openai

In [4]:
%%writefile streamlit_app.py
import streamlit as st
import assemblyai as aai
import openai
import json

aai.settings.api_key = "4e4fb7c0f72d43b2846eb79a61d7da28"

# Replace with your OpenAI API key
openai.api_key = "sk-proj-h3SAhPcKHzPclI6ICkkXT3BlbkFJrwCoYZ0C3NNEYSAC1Y4t"
# Initialize the OpenAI client
# Create the OpenAI client with the API key
client = openai.Client(api_key=openai.api_key )

def get_openAI_response(prompt, model="gpt-3.5-turbo-0125",temperature = 0):
  message = [{'role': 'user', 'content': prompt}]
  response = client.chat.completions.create(
      model=model,
      messages=message,
      temperature=temperature
  )
  return response.choices[0].message.content

# Streamlit app
def main():


    st.sidebar.image("venturedata.jpeg", width=100, use_column_width=False, clamp=True, channels="RGB")
    st.sidebar.title("Audio Analysis")

    uploaded_file = st.file_uploader("Upload an audio file (WAV or MP3)", type=["wav", "mp3"])

    if uploaded_file is not None:
        st.audio(uploaded_file, format='audio/wav' if uploaded_file.type == 'audio/wav' else 'audio/mp3')
        st.sidebar.success("File Uploaded")

        if st.button("Analyse"):

            st.sidebar.success("Analysis Started")
            transcript = aai.Transcriber().transcribe(uploaded_file)
            # Display the transcription
            st.subheader("Transcription:")
            st.markdown(transcript.text)
            st.sidebar.success("Transcription Completed")


            if transcript.text:

                prompt = f"""
                      Given the transcript of a customer service call between a representative and a caller regarding plumbing services, determine whether the call outcome is Booked, Excused, or Unbooked.

                      Please categorize the call based on the actions and outcomes described in the transcript:
                      Definitions:
                      - 'Booked': An appointment or job was successfully booked or an agent was sent to the customer.
                      - 'Unbooked': An appointment or job was finalized or not booked nor an agent was sent to the customer. The callers may need further follow-up or consideration before committing to scheduling a service call or receiving a quote.
                      - 'Not a Lead': The call outcome is Not a Lead because the caller primarily involve existing customers seeking assistance or information related to services provided by Cardinal Plumbing, Heating, and Air, resolving issues, or making personal inquiries, rather than representing potential leads for new business, customer confirming existing appintmemt, or providing information about service procedures without the intent to initiate a new service or expand upon an existing one.
                      - 'Excused': The issue was resolved during the call, no appointment was necessary, and no agent was sent to the customer.

                      Provide your reasoning for your decision in one sentence.

                      Transcript:
                      [The transcript of the call is provided within triple backticks.]

                      ```{transcript.text}```

                      Steps to follow:
                      1. Analyze the transcript and determine the call category. But dont Give output yet.
                      2. Provide "Booked", "Excused", "Not a Lead" or "Unbooked", without any additional text.
                      3. only provide output in json where the first line provide "Class" and the value is "Booked", "Excused", "Not a Lead" or "Unbooked". Second line provide "Explanation" and the value is the reasoning for the decision. Third provide overall "Summary"
                      """
                analysis = get_openAI_response(prompt)
                st.subheader("Analysis:")
                
                data = json.loads(analysis)

                # Access elements separately
                class_value = data["Class"]
                explanation_value = data["Explanation"]
                summary_value = data["Summary"]
                

                st.write(f"\n\n**Class:** \n{class_value}")
                st.write(f"\n\n**Reasoning:** \n{explanation_value}")
                st.write(f"\n\n**Call Summary:** \n{summary_value}\n")
                #st.markdown(explanation_value)
                #st.markdown(summary_value)
                # Determine the color based on the class value
                if class_value == "Booked":
                    st.success("Trigger Sent.")
                else:
                    st.error("Trigger Sent.")
                
                st.sidebar.success("Analysis Completed")


if __name__ == "__main__":
    main()

Overwriting streamlit_app.py
